# Crear de una Base de Datos en Snowflake desde Python

Antes de comenzar hay que tener una cuenta activa en **Snowflake**: https://signup.snowflake.com/  

A continuación tenemos que instalar la librería que nos permite conectar python con el servidor snowflake

```bash
pip install snowflake-connector-python
```

Desde el notebook de python ejecuta:

```python
import snowflake.connector

# Conexión a Snowflake
conn = snowflake.connector.connect(
    user="TU_USUARIO",
    password="TU_CONTRASEÑA",
    account="TU_CUENTA"
)

# Crear un cursor
cur = conn.cursor()
```

Con esto habrás establecido la conexión, ahora vamos a crear la base de datos:

```python
# Ejecutar la consulta para crear la base de datos
cur.execute("CREATE DATABASE mi_base_de_datos;")

print("✅ Base de datos creada exitosamente")

# Cerrar la conexión
cur.close()
conn.close()
```

Ya tienes tu primera base de datos creada en snowflake

```python
from sqlalchemy import create_engine, Column, Integer, String, Date, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database
import pandas as pd
import datetime

# Conectar a Snowflake
USER = "tu_usuario"
PASSWORD = "tu_contraseña"
ACCOUNT = "tu_cuenta"  # ejemplo: "abcd1234.snowflakecomputing.com"
DATABASE = "STOCKIFY"
WAREHOUSE = "COMPUTE_WH"
SCHEMA = "PUBLIC"

engine = create_engine(
    f"snowflake://{USER}:{PASSWORD}@{ACCOUNT}/{DATABASE}/{SCHEMA}?warehouse={WAREHOUSE}"
)

# Crear la base de datos si no existe
if not database_exists(engine.url):
    create_database(engine.url)

Base = declarative_base()

# Definimos las tablas
class Company(Base):
    __tablename__ = 'Company'
    company_code = Column(String(120), primary_key=True)
    security = Column(String(120))
    sec = Column(String(120))
    gics_sector = Column(String(120))
    gics_sub_industry = Column(String(120))
    heads_location = Column(String(120))
    start_date = Column(Date)
    cik = Column(String(120))
    founded = Column(String(120))

class Stocks(Base):
    __tablename__ = 'Stocks'
    stock_id = Column(Integer, primary_key=True)
    company_code = Column(String(120), ForeignKey("Company.company_code"))
    date = Column(Date)
    max_price = Column(Float)
    min_price = Column(Float)
    volume = Column(Float)
    close = Column(Float)
    open = Column(Float)

class user(Base):
    __tablename__ = 'User'
    user_id = Column(Integer,primary_key=True)
    user_name=Column(String(20))
    user_city=Column(String(120))

class transactions(Base):
    __tablename__ = 'Transactions'
    trx_id = Column(Integer,primary_key=True)
    user_id = Column(Integer,ForeignKey("User.user_id"))
    company_code = Column(String(120),ForeignKey("Company.company_code"))
    stock_id = Column(Integer,ForeignKey("Stocks.stock_id"))
    units = Column(Integer)
    
# Hacemos efectiva la creación de las tablas
Base.metadata.create_all(engine)

# Creamos la sesión
Session = sessionmaker(bind=engine)
session = Session()

# cargamos las tablas desde CSV
def populate_table(csv_file,table):

    df = pd.read_csv(csv_file, sep = ';', encoding = 'latin-1')
    df = df.dropna()
    #i=0
    
    for i, val in enumerate(df.values):
        if table == 'Company':
            rec = Company (
                company_code = val[2],
                security = val[3],
                sec = val[4],
                gics_sector = val[5],
                gics_sub_industry = val[6],
                heads_location = val[7],
                start_date = datetime.datetime.strptime(str(val[8]), "%d/%m/%Y").date(),
                cik = val[9],
                founded = val[10]
            )
        
        elif table == 'Stocks':
            rec = Stocks (
                company_code = val[-1],
                date = datetime.datetime.strptime(val[1], "%Y-%m-%d").date(),
                max_price = val[3],
                min_price = val[4],
                volume = val[6],
                close = val[5],
                open = val[2]

            )
        elif table == 'User':
            rec = user (
                user_id = val[0],
                user_name = val[1],
                user_city = val[2]

            )  
        else:
            rec = transactions(
                trx_id = val[0],
                user_id = val[1],
                company_code = val[3],
                stock_id = val[2],
                units = val[4]
                
            )
            
        session.add(rec)
        
    session.commit()
# Poblar la tabla Company desde un CSV
populate_table('./company.csv', 'Company')
```